# The University of Melbourne, School of Computing and Information Systems
# COMP30027 Machine Learning, 2019 Semester 1
-----
## Project 1: Gaining Information about Naive Bayes
-----
###### Student Name(s): Bohan Yang
###### Student ID : 814642
###### Python version: 3
###### Submission deadline: 1pm, Fri 5 Apr 2019

In [47]:
#Explanations of inputs for the functions:
# file is the "name.csv" file name string
# data is the usable format of csv file (output of function preprocess)
# model is a list of four lists: class names, class counts, conditional attributes value counts for each class, 
#   and conditional attributes value probability for each class
# instance is a line of data, without the last column, which is the class


from collections import defaultdict
import numpy as np

This iPython notebook is a template which you may use for your Project 1 submission. (You are not required to use it; in particular, there is no need to use iPython if you do not like it.)

Marking will be applied on the five functions that are defined in this notebook, and to your responses to the questions at the end of this notebook.

You may change the prototypes of these functions, and you may write other functions, according to your requirements. We would appreciate it if the required functions were prominent/easy to find. 

In [48]:
# This function should open a data file in csv, and transform it into a usable format 
def preprocess(file):
    new_data = []
    f = open(file,'r')
    data = f.readlines()
    for line in data:
        new_data.append(line.strip().split(","))
    f.close()
    return new_data

In [49]:
# This function should build a supervised NB model
def train(data):
    class_names = []
    class_count = []
    condi_list = []
    condi_count_list = []
    class_dict = defaultdict(int)
    
    total_count = len(data)
    attr_count = len(data[0])-1
    EPSILON = 1/total_count
    
    #first and second return list: class names and counts 
    for line in data:
        if line[-1] != "?":
            class_dict[line[-1]] += 1
    for i in class_dict.keys():
        class_names.append(i)
    for i in class_dict.values():
        class_count.append(i)
    
    #list of dicts of value counts, and list of dicts of value probs
    for c in class_names:                
        class_condi = []
        condi_count = []
        for a in range(attr_count):
            
            attr_dict = defaultdict(float)
            attr_count_dict = defaultdict(int)
            #smoothing using epsilon
            for line in data:
                attr_count_dict[line[a]] = 0
                attr_dict[line[a]] = EPSILON
            for line in data:
                if (line[-1] == c) & (line[a] != "?"):
                    attr_dict[line[a]] += 1/class_dict[c]
                    attr_count_dict[line[a]] += 1
            class_condi.append(attr_dict)
            condi_count.append(attr_count_dict)
            
        condi_list.append(class_condi)
        condi_count_list.append(condi_count)
        
    model = (class_names, class_count,condi_count_list, condi_list)
    return model        

In [50]:
#an example for running function train
train(preprocess("car.csv"))

(['unacc', 'acc', 'vgood', 'good'],
 [1210, 384, 65, 69],
 [[defaultdict(int, {'vhigh': 360, 'high': 324, 'med': 268, 'low': 258}),
   defaultdict(int, {'vhigh': 360, 'high': 314, 'med': 268, 'low': 268}),
   defaultdict(int, {'2': 326, '3': 300, '4': 292, '5more': 292}),
   defaultdict(int, {'2': 576, '4': 312, 'more': 322}),
   defaultdict(int, {'small': 450, 'med': 392, 'big': 368}),
   defaultdict(int, {'low': 576, 'med': 357, 'high': 277})],
  [defaultdict(int, {'vhigh': 72, 'high': 108, 'med': 115, 'low': 89}),
   defaultdict(int, {'vhigh': 72, 'high': 105, 'med': 115, 'low': 92}),
   defaultdict(int, {'2': 81, '3': 99, '4': 102, '5more': 102}),
   defaultdict(int, {'2': 0, '4': 198, 'more': 186}),
   defaultdict(int, {'small': 105, 'med': 135, 'big': 144}),
   defaultdict(int, {'low': 0, 'med': 180, 'high': 204})],
  [defaultdict(int, {'vhigh': 0, 'high': 0, 'med': 26, 'low': 39}),
   defaultdict(int, {'vhigh': 0, 'high': 13, 'med': 26, 'low': 26}),
   defaultdict(int, {'2': 10,

In [51]:
# This function predicts the class for an instance, based on a trained model 
def predict(model, instance): 
    num_class = len(model[0])
    num_inst = sum(model[1])
    poster_list = []
    final = 0
    for i in range(num_class):
        cond = 0
        for j in range(len(instance)): 
            #use log to avoid underflow
            cond += np.log2(model[3][i][j][instance[j]])
        poster_list.append(np.log2(model[1][i]/num_inst) + cond)  

    for i in range(len(poster_list)):
        if poster_list[i] == max(poster_list):
            final = i
    return model[0][final]

In [52]:
#an example for running function predict
model = train(preprocess("mushroom.csv"))
instance = preprocess("mushroom.csv")[30][:-1]
predict(model, instance)

'e'

In [53]:
#first way to evaluate a Naive Bayes classifier
def accuracy(model, data):
    acc = 0
    for line in data:
        if predict(model, line[:-1]) == line[-1]:
            acc += 1
    print(acc/len(data))

In [54]:
#example for running function accuracy
data = preprocess("mushroom.csv")
model = train(data)
accuracy(model, data)

0.9634416543574594


In [55]:
#second way to evaluate a Naive Bayes classifier 
#This function prints out the TN, TP, FN, FP's of the Naive Bayes prediction. 
#The metrics can be calculated by hand easily with the output
def evaluate(model, data):
    TN = defaultdict(int)
    TP = defaultdict(int)
    FN = defaultdict(int)
    FP = defaultdict(int)
    
    for c in model[0]:
        for line in data:
            if (predict(model, line[:-1]) == c) and (line[-1] == c):
                TP[c] += 1
            elif (predict(model, line[:-1]) != c) and (line[-1] != c):
                TN[c] += 1
            elif (predict(model, line[:-1]) == c) and (line[-1] != c):
                FP[c] += 1
            elif (predict(model, line[:-1]) != c) and (line[-1] == c):    
                FN[c] += 1
    
    print("       TP   TN   FN   FP")
    for attr in TP.keys():
        print(attr,":", TP[attr], TN[attr], FN[attr], FP[attr])


In [56]:
#an example for running function evaluate
data = preprocess("car.csv")
model = train(data)

evaluate(model, data)

       TP   TN   FN   FP
unacc : 1162 433 48 85
acc : 289 1224 95 120
vgood : 37 1661 28 2
good : 21 1647 48 12


In [57]:
# This function should calculate the Information Gain of an attribute or a set of attribute, with respect to the class
def info_gain(model):
    num_inst = sum(model[1])
    class_num = len(model[0])
    attr_num = len(model[2][0])

    #calculate H(R)
    hr = 0
    for c in model[1]:
        hr -= (c/num_inst) * np.log2(c/num_inst)
    
    #calculate p(value) and h(value) for each attribute
    p_list = []
    h_list = []
    total_list = []
    for i in range(attr_num):
        p_value = defaultdict(int)
        t_value = defaultdict(int)
        
        for c in range(class_num):
            
            #total counts for the value
            for k in model[3][c][i].keys():      
                if k not in t_value.keys():
                    t_value[k]= model[2][c][i][k]
                else:
                    t_value[k] += model[2][c][i][k]
            
            #calculate p(value) for each attribute
            for k in model[3][c][i].keys():      
                if k not in p_value.keys():
                    p_value[k]= model[2][c][i][k]/num_inst
                else:
                    p_value[k] += model[2][c][i][k]/num_inst
        
        p_list.append(p_value)
        total_list.append(t_value)

    #calculate h(value) for each attribute
    for i in range(attr_num):
        h_value = defaultdict(float)
        for c in range(class_num):
            for k in model[3][c][i].keys():
                if (model[2][c][i][k]!= 0) and (total_list[i][k] != 0):
                    if k not in h_value.keys():
                        h_value[k] = -((model[2][c][i][k]/total_list[i][k]) * np.log2(model[2][c][i][k]/total_list[i][k]))
                    else:
                        h_value[k] -= (model[2][c][i][k]/total_list[i][k]) * np.log2(model[2][c][i][k]/total_list[i][k])       
        h_list.append(h_value)
        
    #calculate mean information for each attribute
    mean_info = []
    for i in range(attr_num):
        temp = 0
        for k in (p_list[i]).keys():
            temp += (p_list[i][k]) * (h_list[i][k])
        mean_info.append(temp)
        
    #calculate information gain for each attribute
    info_gain = []
    for i in mean_info:
        info_gain.append(hr - i)
        
    return(info_gain)

In [58]:
#an example for running the function info_gain
info_gain(train(preprocess("hepatitis.csv")))

[0.03660746514280977,
 0.015265380561918285,
 0.014490701150154384,
 0.08645063847884216,
 0.08322845589007444,
 0.013806029835453981,
 0.0903522944652434,
 0.09049078626122986,
 0.058739302370822144,
 0.12938741279822152,
 0.151635200236383,
 0.10012174391687256,
 0.08493296456638766]

Questions (you may respond in a cell or cells below):

1. The Naive Bayes classifiers can be seen to vary, in terms of their effectiveness on the given datasets (e.g. in terms of Accuracy). Consider the Information Gain of each attribute, relative to the class distribution — does this help to explain the classifiers’ behaviour? Identify any results that are particularly surprising, and explain why they occur.
2. The Information Gain can be seen as a kind of correlation coefficient between a pair of attributes: when the gain is low, the attribute values are uncorrelated; when the gain is high, the attribute values are correlated. In supervised ML, we typically calculate the Infomation Gain between a single attribute and the class, but it can be calculated for any pair of attributes. Using the pair-wise IG as a proxy for attribute interdependence, in which cases are our NB assumptions violated? Describe any evidence (or indeed, lack of evidence) that this is has some effect on the effectiveness of the NB classifier.
3. Since we have gone to all of the effort of calculating Infomation Gain, we might as well use that as a criterion for building a “Decision Stump” (1-R classifier). How does the effectiveness of this classifier compare to Naive Bayes? Identify one or more cases where the effectiveness is notably different, and explain why.
4. Evaluating the model on the same data that we use to train the model is considered to be a major mistake in Machine Learning. Implement a hold–out or cross–validation evaluation strategy. How does your estimate of effectiveness change, compared to testing on the training data? Explain why. (The result might surprise you!)
5. Implement one of the advanced smoothing regimes (add-k, Good-Turing). Does changing the smoothing regime (or indeed, not smoothing at all) affect the effectiveness of the Naive Bayes classifier? Explain why, or why not.
6. Naive Bayes is said to elegantly handle missing attribute values. For the datasets with missing values, is there any evidence that the performance is different on the instances with missing values, compared to the instances where all of the values are present? Does it matter which, or how many values are missing? Would a imputation strategy have any effect on this?

Don't forget that groups of 1 student should respond to question (1), and one other question of your choosing. Groups of 2 students should respond to question (1) and question (2), and two other questions of your choosing. Your responses should be about 150-250 words each.






Question 1: The Naive Bayes classifiers can be seen to vary, in terms of their effectiveness on the given datasets (e.g. in terms of Accuracy). Consider the Information Gain of each attribute, relative to the class distribution — does this help to explain the classifiers’ behaviour? Identify any results that are particularly surprising, and explain why they occur.

In [76]:
print(sum(info_gain(train(preprocess("anneal.csv"))))/len(info_gain(train(preprocess("anneal.csv")))))
accuracy(train(preprocess("anneal.csv")), preprocess("anneal.csv"))
print(" ")
print(sum(info_gain(train(preprocess("cmc.csv"))))/len(info_gain(train(preprocess("cmc.csv")))))
accuracy(train(preprocess("cmc.csv")), preprocess("cmc.csv"))
print(" ")
print(sum(info_gain(train(preprocess("car.csv"))))/len(info_gain(train(preprocess("car.csv")))))
accuracy(train(preprocess("car.csv")), preprocess("car.csv"))
print(" ")
print(sum(info_gain(train(preprocess("nursery.csv"))))/len(info_gain(train(preprocess("nursery.csv")))))
accuracy(train(preprocess("nursery.csv")), preprocess("nursery.csv"))

0.0882166374169252
0.9576837416481069
 
0.0379949774347087
0.5057705363204344
 
0.11441568230991828
0.8732638888888888
 
0.1614732050742847
0.9030092592592592


Answer for question 1:
Since the accuracy function and train function here do not delete the instances with missing values, the accuracies for predictions of the dataset with missing values are underestimated. Therefore, the datasets with missing values are not taken into consideration in this specific analyzation. As the comparison between average information gain and accuracy of the Naïve Bayes classifier shown above, it appears that in general, higher information gain leads to higher accuracy for the predictions. Since the information gain calculates the reduction of entropy after knowing the attribute value, a higher information gain implies a less random sample after knowing the attribute value. Naïve Bayes classifier predicts classes basing on the probability of a certain class with the attribute values known. Therefore if the sample is less random with the knowledge of the attribute value, it is more predictable, thus leads to a higher Naïve Bayes classifier accuracy. 







Question 3: Since we have gone to all of the effort of calculating Infomation Gain, we might as well use that as a criterion for building a “Decision Stump” (1-R classifier). How does the effectiveness of this classifier compare to Naive Bayes? Identify one or more cases where the effectiveness is notably different, and explain why.


In [60]:
#predict the class of an instance with 1-R decision stump
def decision_stump(model, instance):
    IG = info_gain(model)
    class_num = len(model[0])
    
    #picking the attribute to apply the rule with
    rule_attr = 0
    for i in range(len(IG)):
        if IG[i] == max(IG):
            rule_attr = i
            
    #initializing
    max_count_dict = defaultdict(int)
    for c in range(class_num):
        for k in model[2][c][i].keys():
            max_count_dict[k] = 0
        
    #recording most frequent class for each attribute value    
    for c in range(class_num):
        for k in model[2][c][i].keys():
            if model[2][c][i][k] > max_count_dict[k]:
                max_count_dict[k] = model[2][c][i][k]

    max_class_dict = defaultdict(int)
    for c in range(class_num):
        for k in model[2][c][i].keys():
            if model[2][c][i][k] == max_count_dict[k]:
                max_class_dict[k] = model[0][c]
                
    #predicting for this instance
    value = instance[i]
    return max_class_dict[value]

In [61]:
#print the accuracy of the decision_stump for certain model and data
def accuracy_DS(model, data):
    acc = 0
    for line in data:
        if decision_stump(model, line[:-1]) == line[-1]:
            acc += 1
    print(acc/len(data))

In [62]:
#comparison of the accuracy for the 2 classifiers
print("[data1]")
print("for cmc.csv:")
accuracy_DS(train(preprocess("cmc.csv")), preprocess("cmc.csv"))
accuracy(train(preprocess("cmc.csv")), preprocess("cmc.csv"))
print(" ")
print("for car.csv:")
accuracy_DS(train(preprocess("car.csv")), preprocess("car.csv"))
accuracy(train(preprocess("car.csv")), preprocess("car.csv"))
print(" ")
print("for mushroom.csv:")
accuracy_DS(train(preprocess("mushroom.csv")), preprocess("mushroom.csv"))
accuracy(train(preprocess("mushroom.csv")), preprocess("mushroom.csv"))
print(" ")
print("for primary-tumor.csv:")
accuracy_DS(train(preprocess("primary-tumor.csv")), preprocess("primary-tumor.csv"))
accuracy(train(preprocess("primary-tumor.csv")), preprocess("primary-tumor.csv"))
print(" ")
print("for hepatitis.csv:")
accuracy_DS(train(preprocess("hepatitis.csv")), preprocess("hepatitis.csv"))
accuracy(train(preprocess("hepatitis.csv")), preprocess("hepatitis.csv"))


[data1]
for cmc.csv:
0.4270196877121521
0.5057705363204344
 
for car.csv:
0.7002314814814815
0.8732638888888888
 
for mushroom.csv:
0.690300344657804
0.9634416543574594
 
for primary-tumor.csv:
0.24778761061946902
0.5988200589970502
 
for hepatitis.csv:
0.7935483870967742
0.8516129032258064


Answer for question 3: 
From the comparisons of the accuracy for decision stump and Naïve Bayes shown above[data1], it is shown that decision stump generally has a lower accuracy than Naïve Bayes. This could be because the decision stump only takes one attribute into consideration in the decision-making process, and is not sensitive to the interaction between different attributes. For the datasets that do not contain large number of classes, such as cmc. csv, car. csv and hepatitis. csv, the difference between two accuracies are not too significant. 

However, the decision stump accuracy for primary-tumor. csv is shockingly low. Primary-tumor has more classes than attributes and has a large amount of missing values. This quality has caused low accuracy for both classifiers, but decision stump is extremely low. It indicates that decision stump does not work well when there are too many classes or missing values.  

An example of another unfortunate situation the decision stump could encounter is shown below[data2]. The most frequent classes for each attribute value are the same, resulting in every instance getting classified into the same class. This could be another causation of the low accuracy. 

Another problem which happened to not appear in the test cases, is when the critical attribute value in the testing instance is missing. Then the 1R will not be able to give any class. 


In [63]:
def decision_stump_print(model):
    IG = info_gain(model)
    class_num = len(model[0])
    
    #picking the attribute to apply the rule with
    rule_attr = 0
    for i in range(len(IG)):
        if IG[i] == max(IG):
            rule_attr = i
            
    #initializing
    max_count_dict = defaultdict(int)
    for c in range(class_num):
        for k in model[2][c][i].keys():
            max_count_dict[k] = 0
        
    #recording most frequent class for each attribute value    
    for c in range(class_num):
        for k in model[2][c][i].keys():
            if model[2][c][i][k] > max_count_dict[k]:
                max_count_dict[k] = model[2][c][i][k]

    max_class_dict = defaultdict(int)
    for c in range(class_num):
        for k in model[2][c][i].keys():
            if model[2][c][i][k] == max_count_dict[k]:
                max_class_dict[k] = model[0][c]
                
    print(max_class_dict)

In [64]:
print("data2")
decision_stump_print(train(preprocess("cmc.csv")))

data2
defaultdict(<class 'int'>, {'Good': 'No-use', 'Not-good': 'No-use'})
